Ref: https://github.com/microsoft/ai-agents-for-beginners/blob/main/07-planning-design/code_samples/07-python-agent-framework.ipynb

In [ ]:
# 📦 Import Standard Libraries
# os: For environment variable access
# dotenv: For loading environment variables from .env file
import os
from dotenv import load_dotenv

# 📝 Import Data Modeling Libraries
# Pydantic: For data validation and structured outputs
# BaseModel: Base class for creating data models
# Field: For adding metadata and validation to model fields
# List: Type hint for list collections
from pydantic import BaseModel, Field
from typing import List
# 🤖 Import Microsoft Agent Framework Components
# ChatMessage: For creating structured chat messages
# Role: Enum for message roles (user, assistant, system)
# ChatOptions: Configuration options for chat interactions
# OpenAIChatClient: Client for OpenAI-compatible APIs (GitHub Models)
from agent_framework import ChatMessage, Role, ChatOptions
from agent_framework.openai import OpenAIChatClient
# 🔧 Load Environment Variables
# Load configuration from .env file
# Required: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
load_dotenv()

True

In [2]:
# 🎯 Agent Configuration
# Define the agent's name and role

AGENT_NAME = "TravelAgent"

# 📋 System Instructions for Planning Agent
# This agent acts as a coordinator that decides which specialized agents to use
AGENT_INSTRUCTIONS = """You are a planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialized in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
"""

In [3]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
options = ChatOptions(response_format=TravelPlan)
client = OpenAIChatClient(base_url=os.environ.get("GITHUB_ENDPOINT")
                          , api_key=os.environ.get("GITHUB_TOKEN")
                          , model_id=os.environ.get("GITHUB_MODEL_ID"))
agent =  client.create_agent(name= AGENT_NAME 
                             , instructions=AGENT_INSTRUCTIONS)
messages = [
        ChatMessage(role=Role.USER
                    , text="Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne")
    ]

In [5]:
response = await agent.run(messages,response_format=TravelPlan)
response.messages[0].text

'{"main_task":"Book a family trip from Singapore to Melbourne for 4 people","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Book round-trip flights from Singapore to Melbourne for a family of 4, including 2 adults and 2 kids."},{"assigned_agent":"HotelBooking","task_details":"Find and book a family-friendly hotel in Melbourne that can accommodate 4 people with suitable amenities."},{"assigned_agent":"CarRental","task_details":"Arrange a car rental in Melbourne for the family to explore the city."},{"assigned_agent":"ActivitiesBooking","task_details":"Plan and book family-friendly activities in Melbourne suitable for kids."}]}'

In [8]:
from pprint import pprint
pprint(response.messages[0].text)

('{"main_task":"Book a family trip from Singapore to Melbourne for 4 '
 'people","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Book '
 'round-trip flights from Singapore to Melbourne for a family of 4, including '
 '2 adults and 2 kids."},{"assigned_agent":"HotelBooking","task_details":"Find '
 'and book a family-friendly hotel in Melbourne that can accommodate 4 people '
 'with suitable '
 'amenities."},{"assigned_agent":"CarRental","task_details":"Arrange a car '
 'rental in Melbourne for the family to explore the '
 'city."},{"assigned_agent":"ActivitiesBooking","task_details":"Plan and book '
 'family-friendly activities in Melbourne suitable for kids."}]}')
